# Knapsack

In [8]:
# Import path to source directory (bit of a hack in Jupyter)
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, os.path.join('..', 'src')))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Import external librarires
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Import our code
from objfun_tsp import TSPGrid

## Knapsack problem

### Description

* Input: nonnegative numbers $p_1, \dots, p_n, a_1, \dots, a_n, W$
* The aim is to maximize the sum 

$$\mathrm{max}\sum_{j = 1}^np_jx_j$$

* while the condition holds true:

$$ \sum_{j = 1}^na_jx_j \leq W, $$

* where $x \in \{0,1\}^n$ if we talk about 0/1 Knapsack problem and $x \in (\mathbb{R}_0^+)^n$ if fractional Knapsack problem is considered.

## 0/1 Knapsack problem

* Firstly we aim our attention at 0/1 Knapsack problem. Meaning that only whole items can be added to the Knapsack.

In [13]:
from greedy2 import GreedyAlg2
of = GreedyAlg2()  

In [15]:
value = [50, 40, 30, 50, 30, 24, 36]
weight = [5, 4, 6, 3, 2, 6, 7]
TotalWeight = 20

optimal, Amatrix = of.greedy_0_1_dynamic(TotalWeight, weight, value);
print("Optimal value of the Knapsack is: {:.1f}".format(optimal))


Optimal value of the Knapsack is: 200.0


In [17]:
items = of.BackTracking(TotalWeight, weight, Amatrix)
print("Sorted items to be added to knapsack: {}".format(items))

Sorted items to be added to knapsack: [0, 1, 2, 3, 4]


## Simulated annealing for 0/1 Knapsack problem

### Algorithm

* Take parameter $\beta$
* Choose item randomly from uniform distribution while you find one, that has not been added to the Knapsack yet
* If the weight limit after adding the item to the knapsack is exceeded drop an item from the Knapsack, that has been chosen randomly
* Otherwise add the item to the Knapsack
* This way new sortment is obtained and compute the difference between the sortments
* With probability $P$ determined by $\mathrm{min}\{1, \mathrm{e}^{\beta \Delta} \}$ we accept the new assortment
* Otherwise, old assortment is the new assortment
* Repeat the algorithm $n$ times for all parameters $\beta$

In [18]:
indices = [0,1,2,3,4,5,6]
value = [50, 40, 30, 50, 30, 24, 36]
weight = [5, 4, 6, 3, 2, 6, 7]
TotalWeight = 20
beta = np.linspace(0, 1, 101)
n = 1000

items, optimal = of.Knapsack(value, weight, TotalWeight, beta, n)
print("Optimal value of the Knapsack is: {:.1f}".format(optimal))
print("Items of the Knapsack: {}".format(items))

Optimal value of the Knapsack is: 200.0
Items of the Knapsack: [0, 1, 2, 3, 4]


## Fractional Knapsack problem

* Now we are interested in fractional Knapsack problem. Meaning that fractions of items can be added to the Knapsack.


### Algorithm

* Assume knapsack holds weight $W$ and items have value $value_i$ and weights $weight_i$
* Sort items by value/weight ration: $\frac{value_i}{weight_i}$ in descending order
* Take the item with highest ratio if the weight limit is not exceeded otherwise take $\frac{W-load}{w_i}$ where load is weight of items already added to the knapsack
* Continue unless weight limit or number of items is reached\


In [19]:
value = [50, 40, 30, 50, 30, 24, 36]
weight = [5, 4, 6, 3, 2, 6, 7]
TotalWeight = 20

optimal = of.knapsack_fractional(TotalWeight, weight, value)
print("Optimal value of the Knapsack is: {:.1f}".format(optimal))


Optimal value of the Knapsack is: 200.9


#### The above results suggest that the highest value of the Knapsack was obtained by Fractional Knapsack.  For 0/1 Knapsack both methods (greedy dynamic and simulated annealing) has shown the same result .